In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
# minmaxscaler와도 비슷하지만, outliers에 대해서 더 잘 잡는 robustscaler을 사용해보자.
from sklearn.preprocessing import RobustScaler
#feature 개수가 너무 많으므로 PCA를 활용
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [ ]:
data = pd.read_csv('../input/csgo-round-winner-classification/csgo_round_snapshots.csv')
data

In [ ]:
data.isnull()

In [ ]:
#null값이 0개. 전처리가 사전에 된듯?
#전체 dataframe의 cell에서 null값의 총 개수를 반환
np.sum(np.sum(data.isnull()))

In [ ]:
data.select_dtypes(np.number)

In [ ]:
#흠... 어떻게 drop method가 input으로 dataframe을 받을 수 있지?
data.drop(data.select_dtypes(np.number), axis=1)

In [ ]:
data

In [ ]:
#converting data['bomb_planted']'s true, false value into 0,1
#이렇게 할 수 있는 경우는 기존의 값이 true, false로만 구성되어 있을때
data['bomb_planted'] = data['bomb_planted'].astype(np.int16)

In [ ]:
encoder = LabelEncoder()

data['map'] = encoder.fit_transform(data['map'])
map_mappings={index : label for index, label in enumerate(encoder.classes_)}
print(map_mappings)

In [ ]:
data['round_winner'] = encoder.fit_transform(data['round_winner'])
roundwinner_mappings={index : label for index, label in enumerate(encoder.classes_)}
print(roundwinner_mappings)

In [ ]:
data

In [ ]:
y=data['round_winner']
X=data.drop('round_winner', axis=1)

In [ ]:
#mean을 0으로 scaling
scaler = RobustScaler()
X= scaler.fit_transform(X)
pd.DataFrame(X)

**Features가 너무 많으므로
=>> Correaltion metrics와 PCA사용**

In [ ]:
sns.heatmap(pd.DataFrame(X).corr())

In [ ]:
pca = PCA(n_components=96)
pca.fit(X)

In [ ]:
plt.hist(pca.explained_variance_ratio_, bins=96)
plt.show()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
#alpha값을 제외한 부분까지는 total_variance값에 들어가야 한다.
def getKComponents(pca, alpha):
    total_variance=0
    for feature, variance in enumerate(pca.explained_variance_ratio_):
        total_variance+=variance
        if (total_variance>1-alpha):
            return feature+1
    return pca.explained_variance_ratio_
        

In [ ]:
#96 features가 33 features로 줄음. 95%의 data 설명가
k = getKComponents(pca, 0.05)

In [ ]:
X=pca.transform(X)[:, 0:k]
pd.DataFrame(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [ ]:
log_model = LogisticRegression()
nn_model = MLPClassifier(verbose=True)

log_model.fit(X_train, y_train)
nn_model.fit(X_train, y_train)

In [ ]:
#80%의 예측!! 매우 양호
print(f"Logistic model : {log_model.score(X_test, y_test)}")
print(f"Neural Net model : {nn_model.score(X_test, y_test)}")